In [1]:
import pandas as pd
import requests as r
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta, date
import numpy as np
import time
import os
import asyncio, aiohttp, nest_asyncio
from aiohttp_retry import RetryClient, ExponentialRetry
nest_asyncio.apply()
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:71.0) Gecko/20100101 Firefox/71.0'}

In [2]:
#https://stackoverflow.com/questions/23267409/how-to-implement-retry-mechanism-into-python-requests-library

import logging
import requests

from requests.adapters import HTTPAdapter, Retry

# logging.basicConfig(level=logging.DEBUG)

s = requests.Session()
retries = Retry(total=10, backoff_factor=1, status_forcelist=[ 502, 503, 504 ])
s.mount('http://', HTTPAdapter(max_retries=retries))

pwd = os.getcwd()
if 'L2 TVL' in pwd:
    prepend = ''
else:
    prepend = 'L2 TVL/'

In [3]:
trailing_num_days = 7

start_date = date.today()-timedelta(days=trailing_num_days +1)

# start_date = datetime.strptime('2022-07-13', '%Y-%m-%d').date()

#chains to get
chains_tg = [
        'Ethereum'
        ,'Optimism'
        ,'Binance'
        ,'Arbitrum'
        ,'Polygon'
        ,'Avalanche'
        ]



In [4]:
#get all apps > 10 m tvl
all_api = 'https://api.llama.fi/protocols'
res = pd.DataFrame( r.get(all_api, headers=header).json() )
res = res[res['tvl'] > 0]

print(res.columns)
# Find if the app is in any of our chains
res['is_chain_tg'] = res['chains'].apply(lambda y: any(ch in y for ch in chains_tg) )

# display(res[['name','chains','is_chain_tg']])

res = res[res['is_chain_tg'] == True]
# display(res)

Index(['id', 'name', 'address', 'symbol', 'url', 'referralUrl', 'description',
       'chain', 'logo', 'audits', 'audit_note', 'gecko_id', 'cmcId',
       'category', 'chains', 'module', 'twitter', 'audit_links', 'oracles',
       'slug', 'tvl', 'chainTvls', 'change_1h', 'change_1d', 'change_7d',
       'fdv', 'mcap', 'openSource', 'language', 'staking', 'parentProtocol',
       'pool2', 'forkedFrom', 'listedAt'],
      dtype='object')


In [5]:

protocols = res[['name','slug','category','chainTvls','parentProtocol']]
# print(protocols)
re = res['chainTvls']
# r[1].keys()
protocols['chainTvls'] = protocols['chainTvls'].apply(lambda x: list(x.keys()) )
# protocols[protocols['chainTvls'].map(set(['Arbitrum']).issubset)]
# filter out chains we don't care about
protocols['chainTvls'] = protocols['chainTvls'].apply(lambda y: [x for x in y if x in chains_tg])
#remove apps if all got removed (shouldn't happen, but just to sanity check)
protocols['num_chains'] = protocols['chainTvls'].apply(lambda x: len(x))
protocols = protocols[protocols['num_chains'] > 0]
# protocols['chainTvls'] = [x for x in protocols['chainTvls'] if x not in chains_tg]
# [x for x in l1 if x not in l2]
# [item for i, item in enumerate(x) if x.find(item, i + 1) == -1]
display(protocols)

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_12572/2310541583.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  protocols['chainTvls'] = protocols['chainTvls'].apply(lambda x: list(x.keys()) )
/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_12572/2310541583.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  protocols['chainTvls'] = protocols['chainTvls'].apply(lambda y: [x for x in y if x in chains_tg])
/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_12572/2310541583.p

,name,slug,category,chainTvls,parentProtocol,num_chains
0,MakerDAO,makerdao,CDP,[Ethereum],NaN,1
1,Polygon Bridge & Staking,polygon-bridge-&-staking,Chain,[Polygon],NaN,1
2,Lido,lido,Liquid Staking,[Ethereum],NaN,1
3,Curve,curve,Dexes,"[Optimism, Ethereum, Avalanche, Polygon, Arbit...",NaN,5
4,WBTC,wbtc,Bridge,[Ethereum],NaN,1
...,...,...,...,...,...,...
1768,Dios Finance,dios-finance,Reserve Currency,[Binance],NaN,1
1769,ZeroShift,zeroshift,Reserve Currency,[Avalanche],NaN,1
1770,Asgard Finance,asgard-finance,Yield,[Ethereum],NaN,1
1771,Nidhi-Dao,nidhi-dao,Reserve Currency,[Polygon],NaN,1


In [6]:
# protocols = protocols[ protocols['slug'] == 'uniswap-v3' ]
# api_str = 'https://api.llama.fi/protocol/'
# ad = pd.DataFrame( r.get(api_str).json()['chainTvls'] ).T[['tokens']]
# ad

In [7]:
# api_str = 'https://api.llama.fi/protocol/uniswap'
# prot_req = r.get(api_str, headers=header).json()['chainTvls']
# # prot_req['Ethereum']
# prot_req

In [8]:
statuses = {x for x in range(100, 600)}
statuses.remove(200)
statuses.remove(429)

In [9]:
async def get_tvl(apistring, header, statuses, chains, prot):
        prod = []
        retry_client = RetryClient()
        async with retry_client.get(apistring, retry_options=ExponentialRetry(attempts=10), raise_for_status=statuses) as response:
                try:
                        # print(apistring)
                        prot_req = await response.json()
                        p_name = prot_req['name']
                        p_cat = prot_req['category']
                        try:
                                p_par = prot_req['parentProtocol']
                        except:
                                p_par = prot_req['name']
                        prot_req = prot_req['chainTvls']
                        
                        for ch in chains:
                                ad = pd.json_normalize( prot_req[ch]['tokens'] )
                                ad_usd = pd.json_normalize( prot_req[ch]['tokensInUsd'] )
                        #             ad = ad.merge(how='left')
                                if not ad.empty:
                                        ad = pd.melt(ad,id_vars = ['date'])
                                        ad = ad.rename(columns={'variable':'token','value':'token_value'})
                                        ad_usd = pd.melt(ad_usd,id_vars = ['date'])
                                        ad_usd = ad_usd.rename(columns={'variable':'token','value':'usd_value'})
                                        ad = ad.merge(ad_usd,on=['date','token'])
                                        
                                        ad['date'] = pd.to_datetime(ad['date'], unit ='s') #convert to days
                                        ad['token'] = ad['token'].str.replace('tokens.','', regex=False)
                                        ad['protocol'] = prot
                                        ad['chain'] = ch
                                        ad['name'] = p_name
                                        ad['category'] = p_cat
                                        ad['parentProtocol'] = p_par
                                #         ad['start_date'] = pd.to_datetime(prot[1])
                                        # ad['date'] = ad['date'] - timedelta(days=1) #change to eod vs sod
                                        prod.append(ad)
                except Exception as e:
                        raise Exception("Could not convert json")
        await retry_client.close()
        
        return prod

In [10]:
def get_range(protocols):
        data_dfs = []
        fee_df = []
        # for dt in date_range:
        #         await asyncio.gather()
        #         data_dfs.append(res_df)
        #         # res.columns
        # try:
        #         loop.close()
        # except:
        #         #nothing
        loop = asyncio.get_event_loop()
        #get by app
        api_str = 'https://api.llama.fi/protocol/'
        # print(protocols)
        prod = []
        tasks = []
        for index,proto in protocols.iterrows():
                #     print(proto)
                prot = proto['slug']
                chains = proto['chainTvls']
                apic = api_str + prot
                #     time.sleep(0.1)
                tasks.append( get_tvl(apic, header, statuses, chains, prot) )
        # print(tasks)
        data_dfs = loop.run_until_complete(asyncio.gather(*tasks, return_exceptions=True))
        # print(date_range)
        # loop.close()
        # print(data_dfs)
        # fee_df = pd.concat(data_dfs)
        # return fee_df
        return data_dfs

In [11]:
df_df = get_range(protocols)
# print (typeof(df_df_all) )


In [12]:
df_list = []
for dat in df_df:
        if isinstance(dat,list):
                # print(dat)
                for pt in dat: #each list within the list (i.e. multiple chains)
                        try:
                                tempdf = pd.DataFrame(pt)
                                if not tempdf.empty:
                                        # print(tempdf)
                                        df_list.append(tempdf)
                        except:
                                continue
# df_df_all = pd.DataFrame()
# print(df_list)
df_df_all = pd.concat(df_list)

In [13]:
# df_df_all

In [14]:

# df_df_all = pd.concat(df_df_all)
# print(df_df_all[2])
print("done api")

done api


In [15]:
#filter down a bit so we can do trailing comp w/o doing every row
df_df = df_df_all[df_df_all['date'].dt.date >= start_date-timedelta(days=1) ]

#trailing comp
df_df['last_token_value'] = df_df.groupby(['token','protocol','chain'])['token_value'].shift(1)
#now actually filter
df_df = df_df[df_df['date'].dt.date >= start_date ]


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_12572/3942141483.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_df['last_token_value'] = df_df.groupby(['token','protocol','chain'])['token_value'].shift(1)


In [16]:

data_df = df_df.copy()
data_df['token_value'] = data_df['token_value'].replace(0, np.nan)
# price = usd value / num tokens
data_df['price_usd'] = data_df['usd_value']/data_df['token_value']

data_df.sort_values(by='date',inplace=True)

# net token change
data_df['net_token_flow'] = data_df['token_value'] - data_df['last_token_value']
# net token change * current price
data_df['net_dollar_flow'] = data_df['net_token_flow'] * data_df['price_usd']


data_df = data_df[abs(data_df['net_dollar_flow']) < 50_000_000_000] #50 bil error bar for bad prices
data_df = data_df[~data_df['net_dollar_flow'].isna()]


In [17]:

data_df = df_df.copy()
data_df['token_value'] = data_df['token_value'].replace(0, np.nan)
# price = usd value / num tokens
data_df['price_usd'] = data_df['usd_value']/data_df['token_value']

data_df.sort_values(by='date',inplace=True)

# net token change
data_df['net_token_flow'] = data_df['token_value'] - data_df['last_token_value']
# net token change * current price
data_df['net_dollar_flow'] = data_df['net_token_flow'] * data_df['price_usd']


data_df = data_df[abs(data_df['net_dollar_flow']) < 50_000_000_000] #50 bil error bar for bad prices
data_df = data_df[~data_df['net_dollar_flow'].isna()]


In [18]:
netdf_df = data_df[['date','protocol','chain','net_dollar_flow','usd_value','name','category','parentProtocol']]
# display(netdf_df)
netdf_df = netdf_df.fillna(0)
netdf_df = netdf_df.groupby(['date','protocol','chain','name','category','parentProtocol']).sum(['net_dollar_flow','usd_value']) ##agg by app

#usd_value is the TVL on a given day
netdf_df = netdf_df.groupby(['date','protocol','chain','usd_value','name','category','parentProtocol']).sum(['net_dollar_flow'])
netdf_df.reset_index(inplace=True)

netdf_df['cumul_net_dollar_flow'] = netdf_df[['protocol','chain','net_dollar_flow','name','category','parentProtocol']]\
                                    .groupby(['protocol','chain','name','category','parentProtocol']).cumsum()
netdf_df.reset_index(inplace=True)
netdf_df.drop(columns=['index'],inplace=True)


Aiohttp retry client was not closed
Aiohttp retry client was not closed
Aiohttp retry client was not closed
Aiohttp retry client was not closed
Aiohttp retry client was not closed
Aiohttp retry client was not closed
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fa021c57ca0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fa0382fbca0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fa03829fca0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fa0381b2730>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fa0381892e0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fa01a0634f0>


In [19]:

data_df = df_df.copy()
data_df['token_value'] = data_df['token_value'].replace(0, np.nan)
# price = usd value / num tokens
data_df['price_usd'] = data_df['usd_value']/data_df['token_value']

data_df.sort_values(by='date',inplace=True)

# net token change
data_df['net_token_flow'] = data_df['token_value'] - data_df['last_token_value']
# net token change * current price
data_df['net_dollar_flow'] = data_df['net_token_flow'] * data_df['price_usd']


data_df = data_df[abs(data_df['net_dollar_flow']) < 50_000_000_000] #50 bil error bar for bad prices
data_df = data_df[~data_df['net_dollar_flow'].isna()]


In [20]:
#get latest
netdf_df['rank_desc'] = netdf_df.groupby(['protocol', 'chain'])['date'].\
                            rank(method='dense',ascending=False).astype(int)


In [21]:
summary_df = netdf_df[  ( netdf_df['rank_desc'] == 1 ) &\
                        (~netdf_df['chain'].str.contains('-borrowed')) &\
                        (~netdf_df['chain'].str.contains('-staking')) &\
                        (~netdf_df['chain'].str.contains('-pool2')) &\
                        (~( netdf_df['chain'] == 'treasury') ) &\
                        (~( netdf_df['chain'] == 'borrowed') ) &\
                        (~( netdf_df['chain'] == 'staking') ) &\
                        (~( netdf_df['chain'] == 'pool2') )       
#                         & (~( netdf_df['chain'] == 'Ethereum') )
                        ]
summary_df = summary_df.sort_values(by='cumul_net_dollar_flow',ascending=False)
summary_df['pct_of_tvl'] = 100* summary_df['net_dollar_flow'] / summary_df['usd_value']
summary_df['flow_direction'] = np.where(summary_df['cumul_net_dollar_flow']>=0,1,-1)
summary_df['abs_cumul_net_dollar_flow'] = abs(summary_df['cumul_net_dollar_flow'])
display(summary_df)
summary_df.to_csv('l1l2_currenttvl.csv')

,date,protocol,chain,usd_value,name,category,parentProtocol,net_dollar_flow,cumul_net_dollar_flow,rank_desc,pct_of_tvl,flow_direction,abs_cumul_net_dollar_flow
14265,2022-11-09 18:59:47,arrakis-finance,Ethereum,8.802616e+08,Arrakis Finance,Yield,Arrakis Finance,-1.582993e+05,1.865067e+08,1,-0.017983,1,1.865067e+08
13880,2022-11-09 18:59:35,aura,Ethereum,4.156740e+08,Aura,Yield,Aura,-9.814322e+06,1.717242e+08,1,-2.361062,1,1.717242e+08
14635,2022-11-09 18:59:47,lido,Ethereum,5.522311e+09,Lido,Liquid Staking,Lido,2.130537e+06,1.353240e+08,1,0.038581,1,1.353240e+08
15061,2022-11-09 18:59:47,venus,Binance,7.148099e+08,Venus,Lending,Venus,-4.884402e+06,4.771772e+07,1,-0.683315,1,4.771772e+07
14023,2022-11-09 18:59:35,maple,Ethereum,5.056572e+07,Maple,Undercollateralized Lending,Maple,2.148502e+06,4.765003e+07,1,4.248931,1,4.765003e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14284,2022-11-09 18:59:47,balancer,Ethereum,1.392093e+09,Balancer,Dexes,Balancer,-1.292852e+08,-2.988854e+08,1,-9.287107,-1,2.988854e+08
14588,2022-11-09 18:59:47,instadapp,Ethereum,1.540918e+09,Instadapp,Services,Instadapp,-2.693416e+08,-5.890662e+08,1,-17.479301,-1,5.890662e+08
14368,2022-11-09 18:59:47,convex-finance,Ethereum,2.743326e+09,Convex Finance,Yield,Convex Finance,-2.100188e+08,-7.379187e+08,1,-7.655627,-1,7.379187e+08
14662,2022-11-09 18:59:47,makerdao,Ethereum,6.581437e+09,MakerDAO,CDP,MakerDAO,-2.619083e+08,-8.112015e+08,1,-3.979501,-1,8.112015e+08


In [22]:


# # print(summary_df[summary_df['chain']=='Arbitrum'])
# # print(summary_df[summary_df['chain']=='Optimism'])
# # display(summary_df)
# fig = px.treemap(summary_df[summary_df['abs_cumul_net_dollar_flow'] !=0], \
#                  path=[px.Constant("all"), 'chain', 'protocol'], \
# #                  path=[px.Constant("all"), 'token', 'chain', 'protocol'], \
#                  values='abs_cumul_net_dollar_flow', color='flow_direction'
# #                 ,color_discrete_map={'-1':'red', '1':'green'})
#                 ,color_continuous_scale='Spectral'
#                 )
# # fig.data[0].textinfo = 'label+text+value'
# fig.update_traces(root_color="lightgrey")
# fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
# # fig.update_layout(tickprefix = '$')

# fig.show()
# fig_app = px.treemap(summary_df[summary_df['abs_cumul_net_dollar_flow'] !=0], \
#                 #  path=[px.Constant("all"), 'chain', 'protocol'], \
# #                  path=[px.Constant("all"), 'token', 'chain', 'protocol'], \
#                         path=[px.Constant("all"), 'protocol','chain'], \
#                  values='abs_cumul_net_dollar_flow', color='flow_direction'
# #                 ,color_discrete_map={'-1':'red', '1':'green'})
#                 ,color_continuous_scale='Spectral'
#                 )
# # fig.data[0].textinfo = 'label+text+value'
# fig_app.update_traces(root_color="lightgrey")
# fig_app.update_layout(margin = dict(t=50, l=25, r=25, b=25))

In [23]:
# fig.write_image(prepend + "img_outputs/svg/net_app_flows.svg") #prepend + 
# fig.write_image(prepend + "img_outputs/png/net_app_flows.png") #prepend + 
# fig.write_html(prepend + "img_outputs/net_app_flows.html", include_plotlyjs='cdn')

# fig_app.write_image(prepend + "img_outputs/svg/net_app_flows_by_app.svg") #prepend + 
# fig_app.write_image(prepend + "img_outputs/png/net_app_flows_by_app.png") #prepend + 
# fig_app.write_html(prepend + "img_outputs/net_app_flows_by_app.html", include_plotlyjs='cdn')


In [24]:
# ! jupyter nbconvert --to python total_app_net_flows_async.ipynb